<a href="https://colab.research.google.com/github/JinNakagawa/Week8-Assignment/blob/main/Group_13_Final_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
This notebook has two main parts.

In the first part, I use Python to download 2024 MLB data directly from the MLB Stats API and reshape it into analysis-ready tables. I build team-level and player-level datasets (games, team stats, hitters, and pitchers), then store all of them in a SQLite database (`mlb_2024_sql.db`).

In the second part, starting from the **“SQL Analysis”** section, I connect to the SQLite database and run ten structured SQL queries. These queries are designed to (i) measure league-wide home-field advantage, (ii) relate team performance to key metrics such as OPS, ERA, and run differential, and (iii) link individual player quality (hitters and pitchers) to team win rates. Each query includes comments explaining what it does, how it is constructed, and why the result matters for understanding or predicting team performance.

In [1]:
# Imports, constants, helper for MLB API

import requests
import sqlite3
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 3)

MLB_API_BASE   = "https://statsapi.mlb.com/api/v1"
MLB_SEASON_YEAR = 2024
MLB_START_DATE  = "2024-03-20"
MLB_END_DATE    = "2024-10-01"

def mlb_api_get(endpoint, params=None):
    """
    Simple wrapper for MLB Stats API.
    """
    if params is None:
        params = {}
    if not endpoint.startswith("/"):
        endpoint = "/" + endpoint
    url = MLB_API_BASE + endpoint
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Quick connection test
test_call = mlb_api_get("/sports")
print("API connected successfully!") if test_call else print("Check API connection.")

API connected successfully!


In [2]:
# Teams and game-level data (teams, schedule, games)

# Teams table (basic team info)
teams_json = mlb_api_get("/teams", {
    "sportId": 1,    # MLB
    "season": MLB_SEASON_YEAR,
    "activeStatus": "Y"
})

mlb_teams_2024 = pd.json_normalize(teams_json["teams"])[
    ["id", "name", "abbreviation", "teamName", "franchiseName", "venue.name"]
].rename(columns={
    "id": "team_id",
    "name": "team_name",
    "abbreviation": "team_abbr",
    "venue.name": "venue_name"
})

print(f"Number of active MLB teams: {mlb_teams_2024['team_id'].nunique()}")

Number of active MLB teams: 30


In [3]:
mlb_teams_2024.head()

,team_id,team_name,team_abbr,teamName,franchiseName,venue_name
0,133,Oakland Athletics,OAK,Athletics,Oakland,Oakland Coliseum
1,134,Pittsburgh Pirates,PIT,Pirates,Pittsburgh,PNC Park
2,135,San Diego Padres,SD,Padres,San Diego,Petco Park
3,136,Seattle Mariners,SEA,Mariners,Seattle,T-Mobile Park
4,137,San Francisco Giants,SF,Giants,San Francisco,Oracle Park


In [4]:
# Raw schedule for the 2024 regular season
schedule_json = mlb_api_get("/schedule", {
    "sportId": 1,
    "startDate": MLB_START_DATE,
    "endDate": MLB_END_DATE,
    "gameType": "R"
})

records = []
for day in schedule_json.get("dates", []):
    for g in day.get("games", []):
        records.append({
            "game_id": g.get("gamePk"),
            "date": g.get("officialDate"),
            "status": (g.get("status") or {}).get("detailedState"),
            "venue_name": (g.get("venue") or {}).get("name"),
            "home_team_id": ((g.get("teams") or {}).get("home") or {}).get("team", {}).get("id"),
            "home_team": ((g.get("teams") or {}).get("home") or {}).get("team", {}).get("name"),
            "home_score": ((g.get("teams") or {}).get("home") or {}).get("score"),
            "away_team_id": ((g.get("teams") or {}).get("away") or {}).get("team", {}).get("id"),
            "away_team": ((g.get("teams") or {}).get("away") or {}).get("team", {}).get("name"),
            "away_score": ((g.get("teams") or {}).get("away") or {}).get("score")
        })

schedule_raw_2024 = pd.DataFrame.from_records(records)
print(f"Total games retrieved: {len(schedule_raw_2024)}")


Total games retrieved: 2469


In [5]:
# Clean games table (filter MLB vs MLB, add flags and basic metrics)
mlb_ids = set(mlb_teams_2024["team_id"])
team_games_2024 = schedule_raw_2024[
    schedule_raw_2024["home_team_id"].isin(mlb_ids) &
    schedule_raw_2024["away_team_id"].isin(mlb_ids)
].copy()

In [6]:
team_games_2024.head()

,game_id,date,status,venue_name,home_team_id,home_team,home_score,away_team_id,away_team,away_score
0,745444,2024-03-20,Final,Gocheok Sky Dome,135,San Diego Padres,2.0,119,Los Angeles Dodgers,5.0
1,746175,2024-03-21,Final,Gocheok Sky Dome,119,Los Angeles Dodgers,11.0,135,San Diego Padres,15.0
2,747060,2024-03-28,Final,Oriole Park at Camden Yards,110,Baltimore Orioles,11.0,108,Los Angeles Angels,3.0
3,746737,2024-03-28,Final,Great American Ball Park,113,Cincinnati Reds,8.0,120,Washington Nationals,2.0
4,745445,2024-03-28,Final,Petco Park,135,San Diego Padres,6.0,137,San Francisco Giants,4.0


In [7]:
team_games_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   game_id       2469 non-null   int64  
 1   date          2469 non-null   object 
 2   status        2469 non-null   object 
 3   venue_name    2469 non-null   object 
 4   home_team_id  2469 non-null   int64  
 5   home_team     2469 non-null   object 
 6   home_score    2432 non-null   float64
 7   away_team_id  2469 non-null   int64  
 8   away_team     2469 non-null   object 
 9   away_score    2432 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 193.0+ KB


In [8]:
# Set final flag for SQL
team_games_2024["is_final"] = team_games_2024["status"].str.contains("Final", case=False, na=False)
team_games_2024["is_postponed"] = team_games_2024["status"].str.contains(
    "Postponed|Suspended|Delayed Start", case=False, na=False
)

Set game level flag for analysis

In [9]:
# 1 if home team scored more than away team, else 0 (nullable Int)
team_games_2024["home_win"] = (
    team_games_2024["home_score"] > team_games_2024["away_score"]
).astype("Int64")

In [10]:
# 1 if away team scored more than home team, else 0 (nullable Int)
team_games_2024["away_win"] = (
    team_games_2024["away_score"] > team_games_2024["home_score"]
).astype("Int64")

In [11]:
# Total runs scored in each game (home + away)
team_games_2024["total_runs"] = team_games_2024[["home_score", "away_score"]].sum(
    axis=1, min_count=1
)

In [12]:
# Run differential from the home team's perspective (home - away)
team_games_2024["home_run_diff"] = (
    team_games_2024["home_score"] - team_games_2024["away_score"]
)

In [13]:
print("team_games_2024 shape:", team_games_2024.shape)

team_games_2024 shape: (2469, 16)


In [14]:
team_games_2024.head()

,game_id,date,status,venue_name,home_team_id,home_team,home_score,away_team_id,away_team,away_score,is_final,is_postponed,home_win,away_win,total_runs,home_run_diff
0,745444,2024-03-20,Final,Gocheok Sky Dome,135,San Diego Padres,2.0,119,Los Angeles Dodgers,5.0,True,False,0,1,7.0,-3.0
1,746175,2024-03-21,Final,Gocheok Sky Dome,119,Los Angeles Dodgers,11.0,135,San Diego Padres,15.0,True,False,0,1,26.0,-4.0
2,747060,2024-03-28,Final,Oriole Park at Camden Yards,110,Baltimore Orioles,11.0,108,Los Angeles Angels,3.0,True,False,1,0,14.0,8.0
3,746737,2024-03-28,Final,Great American Ball Park,113,Cincinnati Reds,8.0,120,Washington Nationals,2.0,True,False,1,0,10.0,6.0
4,745445,2024-03-28,Final,Petco Park,135,San Diego Padres,6.0,137,San Francisco Giants,4.0,True,False,1,0,10.0,2.0


Prepare sub tables for SQL

In [15]:
# Aggregate stats when each team is playing at home
home_summary = (
    team_games_2024.groupby("home_team")
    .agg(
        games_played=("game_id", "count"),          # number of home games
        home_win_rate=("home_win", "mean"),         # average of 0/1 -> win rate
        avg_home_runs=("home_score", "mean"),       # average runs scored at home
        avg_away_runs_allowed=("away_score", "mean"),  # average runs allowed at home
        pct_postponed=("is_postponed", "mean"),     # share of postponed/suspended games
    )
    .reset_index()
)

In [16]:
home_summary

,home_team,games_played,home_win_rate,avg_home_runs,avg_away_runs_allowed,pct_postponed
0,Arizona Diamondbacks,81,0.543,5.642,5.086,0.000
1,Atlanta Braves,88,0.523,3.901,3.889,0.080
2,Baltimore Orioles,82,0.537,4.630,4.321,0.012
3,Boston Red Sox,83,0.458,4.476,4.890,0.012
4,Chicago Cubs,82,0.537,3.938,3.407,0.012
5,Chicago White Sox,86,0.267,3.085,4.744,0.047
6,Cincinnati Reds,82,0.476,4.407,4.778,0.012
7,Cleveland Guardians,84,0.595,4.675,3.987,0.036
8,Colorado Rockies,82,0.451,4.889,5.827,0.012
9,Detroit Tigers,84,0.512,4.086,4.037,0.036


In [17]:
# Aggregate stats when each team is playing away
away_summary = (
    team_games_2024.groupby("away_team")
    .agg(
        games_played=("game_id", "count"),              # number of away games
        away_win_rate=("away_win", "mean"),             # away win rate
        avg_away_runs=("away_score", "mean"),           # average runs scored away
        avg_home_runs_allowed=("home_score", "mean"),   # average runs allowed when away
    )
    .reset_index()
)

In [18]:
# Merge home and away summaries into a single table
team_summary_2024 = home_summary.merge(
    away_summary,
    left_on="home_team",
    right_on="away_team",
    suffixes=("_home", "_away"),
)

# Use a single team name column; drop redundant one
team_summary_2024.rename(columns={"home_team": "team_name"}, inplace=True)
team_summary_2024.drop(columns=["away_team"], inplace=True)

In [19]:
team_summary_2024

,team_name,games_played_home,home_win_rate,avg_home_runs,avg_away_runs_allowed,pct_postponed,games_played_away,away_win_rate,avg_away_runs,avg_home_runs_allowed
0,Arizona Diamondbacks,81,0.543,5.642,5.086,0.000,82,0.549,5.296,4.642
1,Atlanta Braves,88,0.523,3.901,3.889,0.080,84,0.512,4.790,3.605
2,Baltimore Orioles,82,0.537,4.630,4.321,0.012,82,0.573,5.024,4.293
3,Boston Red Sox,83,0.458,4.476,4.890,0.012,81,0.531,4.753,4.321
4,Chicago Cubs,82,0.537,3.938,3.407,0.012,82,0.476,5.148,4.852
5,Chicago White Sox,86,0.267,3.085,4.744,0.047,81,0.222,3.148,5.272
6,Cincinnati Reds,82,0.476,4.407,4.778,0.012,83,0.458,4.222,3.790
7,Cleveland Guardians,84,0.595,4.675,3.987,0.036,82,0.512,4.123,3.728
8,Colorado Rockies,82,0.451,4.889,5.827,0.012,81,0.296,3.531,5.642
9,Detroit Tigers,84,0.512,4.086,4.037,0.036,83,0.518,4.333,3.889


Team-level hitting, pitching, and fielding stats

In [20]:
def coerce_numeric_cols(table, cols):
    """
    Convert selected columns to numeric dtypes (invalid values -> NaN).
    Useful before doing arithmetic or correlations.
    """
    for c in cols:
        if c in table.columns:
            table[c] = pd.to_numeric(table[c], errors="coerce")
    return table

In [21]:
# Pull team hitting stats for the season
hitting_json = mlb_api_get("/teams/stats", {
    "season": MLB_SEASON_YEAR,
    "group": "hitting",
    "sportIds": 1,
})

In [22]:
# Flatten JSON and select key hitting columns
hitting_stats_2024 = pd.json_normalize(hitting_json["stats"][0]["splits"])[
    ["team.id", "team.name", "stat.avg", "stat.ops", "stat.homeRuns", "stat.runs"]
].rename(
    columns={
        "team.id": "team_id",
        "team.name": "team_name",
        "stat.avg": "AVG",
        "stat.ops": "OPS",
        "stat.homeRuns": "HR",
        "stat.runs": "Runs",
    }
)

In [23]:
# Pull team pitching stats for the season
pitching_json = mlb_api_get("/teams/stats", {
    "season": MLB_SEASON_YEAR,
    "group": "pitching",
    "sportIds": 1,
})

In [24]:
# Flatten JSON and select key pitching columns
pitching_stats_2024 = pd.json_normalize(pitching_json["stats"][0]["splits"])[
    ["team.id", "team.name", "stat.era", "stat.whip", "stat.strikeOuts", "stat.hits", "stat.runs"]
].rename(
    columns={
        "team.id": "team_id",
        "team.name": "team_name",
        "stat.era": "ERA",
        "stat.whip": "WHIP",
        "stat.strikeOuts": "SO",
        "stat.hits": "Hits_Allowed",
        "stat.runs": "Runs_Allowed",
    }
)

In [25]:
# Pull team fielding stats for the season
fielding_json = mlb_api_get("/teams/stats", {
    "season": MLB_SEASON_YEAR,
    "group": "fielding",
    "sportIds": 1,
})

In [26]:
# Flatten JSON and select key fielding columns
fielding_stats_2024 = pd.json_normalize(fielding_json["stats"][0]["splits"])[
    ["team.id", "team.name", "stat.fielding", "stat.errors", "stat.putOuts", "stat.assists"]
].rename(
    columns={
        "team.id": "team_id",
        "team.name": "team_name",
        "stat.fielding": "FieldingPct",
        "stat.errors": "Errors",
        "stat.putOuts": "Putouts",
        "stat.assists": "Assists",
    }
)

In [27]:
# Ensure numeric types for metrics before analysis
hitting_stats_2024 = coerce_numeric_cols(hitting_stats_2024, ["AVG", "OPS", "HR", "Runs"])
pitching_stats_2024 = coerce_numeric_cols(
    pitching_stats_2024, ["ERA", "WHIP", "SO", "Hits_Allowed", "Runs_Allowed"]
)
fielding_stats_2024 = coerce_numeric_cols(
    fielding_stats_2024, ["FieldingPct", "Errors", "Putouts", "Assists"]
)

In [28]:
team_summary_2024["overall_win_rate"] = team_summary_2024[["home_win_rate", "away_win_rate"]].mean(axis=1)

team_summary_2024["run_diff_per_game"] = (
    (team_summary_2024["avg_home_runs"] + team_summary_2024["avg_away_runs"])
    - (team_summary_2024["avg_home_runs_allowed"] + team_summary_2024["avg_away_runs_allowed"])
)

In [29]:
# Merge hitting, pitching, fielding, and win-rate data into a single team-level table
team_stats_full_2024 = (
    hitting_stats_2024
    .merge(pitching_stats_2024, on="team_name")  # join on team_name for hitting + pitching
    .merge(fielding_stats_2024, on="team_name")  # add fielding metrics
    .merge(
        team_summary_2024[["team_name", "overall_win_rate", "run_diff_per_game"]],
        on="team_name",
        how="left",
    )  # attach win rate and run differential
)

print("team_stats_full_2024 shape:", team_stats_full_2024.shape)
team_stats_full_2024.head()

team_stats_full_2024 shape: (30, 19)


,team_id_x,team_name,AVG,OPS,HR,Runs,team_id_y,ERA,WHIP,SO,Hits_Allowed,Runs_Allowed,team_id,FieldingPct,Errors,Putouts,Assists,overall_win_rate,run_diff_per_game
0,135,San Diego Padres,0.263,0.744,190,760,135,3.86,1.22,1453,1296,669,135,0.987,75,4318,1297,0.57,1.123
1,109,Arizona Diamondbacks,0.263,0.777,211,886,109,4.62,1.35,1313,1468,788,109,0.989,62,4330,1456,0.546,1.210
2,117,Houston Astros,0.262,0.740,190,740,117,3.74,1.24,1479,1238,649,117,0.985,84,4296,1312,0.543,1.126
3,119,Los Angeles Dodgers,0.258,0.781,233,842,119,3.90,1.23,1390,1273,686,119,0.985,88,4337,1366,0.601,1.926
4,143,Philadelphia Phillies,0.257,0.750,198,784,143,3.85,1.24,1433,1339,671,143,0.986,85,4328,1467,0.582,1.395


In [30]:
team_stats_full_2024

,team_id_x,team_name,AVG,OPS,HR,Runs,team_id_y,ERA,WHIP,SO,Hits_Allowed,Runs_Allowed,team_id,FieldingPct,Errors,Putouts,Assists,overall_win_rate,run_diff_per_game
0,135,San Diego Padres,0.263,0.744,190,760,135,3.86,1.22,1453,1296,669,135,0.987,75,4318,1297,0.57,1.123
1,109,Arizona Diamondbacks,0.263,0.777,211,886,109,4.62,1.35,1313,1468,788,109,0.989,62,4330,1456,0.546,1.210
2,117,Houston Astros,0.262,0.740,190,740,117,3.74,1.24,1479,1238,649,117,0.985,84,4296,1312,0.543,1.126
3,119,Los Angeles Dodgers,0.258,0.781,233,842,119,3.90,1.23,1390,1273,686,119,0.985,88,4337,1366,0.601,1.926
4,143,Philadelphia Phillies,0.257,0.750,198,784,143,3.85,1.24,1433,1339,671,143,0.986,85,4328,1467,0.582,1.395
5,111,Boston Red Sox,0.252,0.742,194,751,111,4.04,1.26,1353,1363,747,111,0.981,115,4358,1449,0.494,0.017
6,110,Baltimore Orioles,0.250,0.750,235,786,110,3.94,1.24,1380,1303,699,110,0.986,81,4326,1336,0.555,1.040
7,158,Milwaukee Brewers,0.248,0.729,177,777,158,3.65,1.23,1373,1289,641,158,0.985,89,4338,1348,0.567,1.679
8,147,New York Yankees,0.248,0.762,237,815,147,3.74,1.24,1457,1272,668,147,0.984,93,4358,1320,0.57,1.815
9,118,Kansas City Royals,0.248,0.709,170,735,118,3.76,1.24,1339,1303,644,118,0.985,85,4284,1423,0.522,1.123


Make hitters and pitchers df

In [31]:
# ============================================================
# Individual-level (player) stats: hitters & pitchers
# ============================================================

# --- Pull hitters ---
hitters_json = mlb_api_get("/stats", {
    "stats": "season",
    "group": "hitting",
    "season": MLB_SEASON_YEAR,
    "sportIds": 1,
    "gameType": "R",
    "playerPool": "ALL",
    "limit": 10000
})

hitters_raw = pd.json_normalize(hitters_json["stats"][0]["splits"])

# Select columns
hit_cols = [
    "player.fullName", "team.name",
    "stat.gamesPlayed", "stat.atBats", "stat.hits",
    "stat.homeRuns", "stat.runs", "stat.rbi",
    "stat.avg", "stat.obp", "stat.slg", "stat.ops"
]

hitters = hitters_raw[hit_cols].copy()
hitters.columns = [
    "Player", "Team", "Games", "AB", "Hits",
    "HR", "Runs", "RBI",
    "AVG", "OBP", "SLG", "OPS"
]

# Convert numeric columns
for c in ["Games","AB","Hits","HR","Runs","RBI","AVG","OBP","SLG","OPS"]:
    hitters[c] = pd.to_numeric(hitters[c], errors="coerce")

# Qualified hitters: AB >= 200
qualified_hitters = hitters[hitters["AB"] >= 200].copy()
qualified_hitters = qualified_hitters.rename(columns={"Team": "team"})


# --- Pull pitchers ---
pitchers_json = mlb_api_get("/stats", {
    "stats": "season",
    "group": "pitching",
    "season": MLB_SEASON_YEAR,
    "sportIds": 1,
    "gameType": "R",
    "playerPool": "ALL",
    "limit": 10000
})

pitchers_raw = pd.json_normalize(pitchers_json["stats"][0]["splits"])

pitch_cols = [
    "player.fullName", "team.name",
    "stat.gamesPlayed", "stat.inningsPitched",
    "stat.era", "stat.whip",
    "stat.strikeOuts", "stat.baseOnBalls"
]

pitchers = pitchers_raw[pitch_cols].copy()
pitchers.columns = ["Player","Team","Games","Innings","ERA","WHIP","K","BB"]

for c in ["Games","Innings","ERA","WHIP","K","BB"]:
    pitchers[c] = pd.to_numeric(pitchers[c], errors="coerce")

# Qualified pitchers: IP >= 50
qualified_pitchers = pitchers[pitchers["Innings"] >= 50].copy()
qualified_pitchers = qualified_pitchers.rename(columns={"Team": "team"})


# --- Merge player stats with team win rate ---
hitters_team = qualified_hitters.merge(
    team_summary_2024[["team_name","overall_win_rate"]],
    left_on="team",
    right_on="team_name",
    how="left"
).drop(columns=["team_name"])

pitchers_team = qualified_pitchers.merge(
    team_summary_2024[["team_name","overall_win_rate"]],
    left_on="team",
    right_on="team_name",
    how="left"
).drop(columns=["team_name"])

print("hitters_team shape:", hitters_team.shape)
print("pitchers_team shape:", pitchers_team.shape)

hitters_team.head()


hitters_team shape: (351, 13)
pitchers_team shape: (351, 9)


,Player,team,Games,AB,Hits,HR,Runs,RBI,AVG,OBP,SLG,OPS,overall_win_rate
0,Jose Iglesias,New York Mets,85,270,91,4,39,26,0.337,0.381,0.448,0.829,0.524
1,Bobby Witt Jr.,Kansas City Royals,161,636,211,32,125,109,0.332,0.389,0.588,0.977,0.522
2,Xavier Edwards,Miami Marlins,70,265,87,1,39,26,0.328,0.397,0.423,0.820,0.38
3,Vladimir Guerrero Jr.,Toronto Blue Jays,159,616,199,30,98,103,0.323,0.396,0.544,0.940,0.458
4,Aaron Judge,New York Yankees,158,559,180,58,122,144,0.322,0.458,0.701,1.159,0.57


# SQL Analysis

Create SQLite DB and write all tables


In [32]:
import sqlite3

# Create (or connect to) SQLite database
db_path = "mlb_2024_sql.db"
conn_db = sqlite3.connect(db_path)

In [33]:
# Dictionary of dataframes to write into DB
tables_to_write = {
    "teams": mlb_teams_2024,
    "games": team_games_2024,
    "team_summary": team_summary_2024,
    "team_stats": team_stats_full_2024,
    "hitters": hitters_team,
    "pitchers": pitchers_team,
}

In [34]:
# Write each DataFrame into SQLite
for name, df in tables_to_write.items():
    df.to_sql(name, conn_db, if_exists="replace", index=False)
    print(f"Created / replaced table: {name}")

Created / replaced table: teams
Created / replaced table: games
Created / replaced table: team_summary
Created / replaced table: team_stats
Created / replaced table: hitters
Created / replaced table: pitchers


In [35]:
# Connect to the SQLite database you created earlier
conn = sqlite3.connect("mlb_2024_sql.db")

In [36]:
def run_sql(query: str) -> pd.DataFrame:
    """
    Run a SQL query against the SQLite database and return a DataFrame.
    """
    return pd.read_sql_query(query, conn)

# Query 1: League-wide home vs away win rate

In [37]:
q1 = """
SELECT
    AVG(CAST(home_win AS FLOAT)) AS league_home_win_rate,
    AVG(CAST(away_win AS FLOAT)) AS league_away_win_rate
FROM games
WHERE is_final = 1;
"""

df_q1 = run_sql(q1)
df_q1

,league_home_win_rate,league_away_win_rate
0,0.521,0.479


Purpose: Measure the league-wide home and away win rates in the 2024 MLB season to establish a baseline level of home-field advantage.

Across all finalized games in 2024, home teams won about 52% of the time, while away teams won roughly 48%. This confirms a modest but meaningful home-field advantage in MLB, which serves as a reference point for evaluating individual team performance later in the analysis.

# Query 2: Teams Ranked by Win Rate with OPS and ERA
Identify which teams performed best in 2024 and examine how their offensive (OPS) and pitching (ERA) metrics align with overall win rates.


In [38]:
q2 = """
SELECT
    ts.team_name,
    t.team_abbr,
    ts.OPS,
    ts.ERA,
    ts.FieldingPct,
    ts.run_diff_per_game,
    ts.overall_win_rate
FROM team_stats AS ts
JOIN teams AS t
    ON ts.team_name = t.team_name
ORDER BY ts.overall_win_rate DESC;
"""

df_q2 = run_sql(q2)
df_q2.head(10)  # show top 10 teams

,team_name,team_abbr,OPS,ERA,FieldingPct,run_diff_per_game,overall_win_rate
0,Los Angeles Dodgers,LAD,0.781,3.90,0.985,1.926,0.601
1,Philadelphia Phillies,PHI,0.750,3.85,0.986,1.395,0.582
2,San Diego Padres,SD,0.744,3.86,0.987,1.123,0.570
3,New York Yankees,NYY,0.762,3.74,0.984,1.815,0.570
4,Milwaukee Brewers,MIL,0.729,3.65,0.985,1.679,0.567
5,Baltimore Orioles,BAL,0.750,3.94,0.986,1.040,0.555
6,Cleveland Guardians,CLE,0.702,3.61,0.985,1.083,0.554
7,Arizona Diamondbacks,AZ,0.777,4.62,0.989,1.210,0.546
8,Houston Astros,HOU,0.740,3.74,0.985,1.126,0.543
9,New York Mets,NYM,0.734,3.96,0.984,0.877,0.524


Teams with higher overall win rates generally show strong performance on both sides of the ball. The top-performing clubs combine above-average OPS with solid pitching (low ERA) and positive run differentials. This alignment suggests that successful teams in 2024 tended to be well-rounded rather than relying solely on offense or pitching. Additionally, the presence of high fielding percentages among these teams implies that defensive quality also contributed meaningfully to overall success.

# Query 3: Home advantage by team

In [39]:
q3 = """
SELECT
    team_name,
    home_win_rate,
    away_win_rate,
    (home_win_rate - away_win_rate) AS home_advantage  -- home vs away difference
FROM team_summary
ORDER BY home_advantage DESC;
"""

df_q3 = run_sql(q3)
df_q3.head(10)  # show top 10 teams by home advantage
df_q3

,team_name,home_win_rate,away_win_rate,home_advantage
0,Seattle Mariners,0.605,0.439,0.166
1,Colorado Rockies,0.451,0.296,0.155
2,Philadelphia Phillies,0.659,0.506,0.152
3,Texas Rangers,0.543,0.422,0.122
4,Oakland Athletics,0.469,0.378,0.091
5,Cleveland Guardians,0.595,0.512,0.083
6,Los Angeles Dodgers,0.642,0.561,0.081
7,Washington Nationals,0.469,0.398,0.072
8,Kansas City Royals,0.556,0.488,0.067
9,Chicago Cubs,0.537,0.476,0.061


This query compares each team’s home and away win rates and ranks them by the difference between the two. By focusing on home_win_rate - away_win_rate, it highlights which clubs benefited the most from playing at home and which teams performed relatively better on the road. This helps us move from league-wide home advantage (Query 1) to team-specific patterns and gives context for later analyses linking performance to team characteristics.

# Query 4: High-scoring home ballparks (JOIN + GROUP BY)

In [40]:
q4 = """
SELECT
    t.venue_name,
    COUNT(*) AS games_played,
    AVG(g.total_runs) AS avg_total_runs
FROM games AS g
JOIN teams AS t
    ON g.home_team_id = t.team_id
WHERE g.is_final = 1
GROUP BY t.venue_name
HAVING games_played >= 40
ORDER BY avg_total_runs DESC
LIMIT 10;
"""

df_q4 = run_sql(q4)
df_q4

,venue_name,games_played,avg_total_runs
0,Chase Field,81,10.728
1,Coors Field,81,10.716
2,loanDepot park,81,10.272
3,Fenway Park,82,9.366
4,Target Field,80,9.363
5,Yankee Stadium,80,9.238
6,Great American Ball Park,81,9.185
7,Kauffman Stadium,80,9.113
8,Dodger Stadium,81,9.099
9,Citizens Bank Park,81,9.037


Query 4 examines which home ballparks saw the highest run scoring in 2024.
It joins the games table to the teams table on
**home_team_id = team_id**
 then groups by ballpark (venue_name) and computes the average total runs per finalized game.
I restrict the sample to ballparks with at least 40 home games to avoid small-sample noise and then rank venues by average total runs, which helps identify the most offense-friendly environments in the league.

# Query 5: Number of Qualified Hitters per Team (JOIN + Aggregation)
Measure how many qualified hitters (AB ≥ 200) each team has and examine how this relates to team-level performance metrics such as OPS, ERA, and win rate. This helps identify whether deeper lineups are associated with stronger overall results.

In [41]:
q5 = """
WITH hitter_counts AS (
    SELECT
        team,
        COUNT(*) AS qualified_hitters
    FROM hitters
    GROUP BY team
)

SELECT
    ts.team_name,
    hc.qualified_hitters,
    ts.OPS,
    ts.ERA,
    ts.overall_win_rate
FROM team_stats AS ts
LEFT JOIN hitter_counts AS hc
    ON ts.team_name = hc.team
ORDER BY
    hc.qualified_hitters DESC,
    ts.overall_win_rate DESC;
"""

df_q5 = run_sql(q5)
df_q5.head(15)

,team_name,qualified_hitters,OPS,ERA,overall_win_rate
0,Kansas City Royals,15,0.709,3.76,0.522
1,Atlanta Braves,15,0.724,3.49,0.517
2,Minnesota Twins,15,0.726,4.26,0.491
3,San Diego Padres,14,0.744,3.86,0.570
4,Cincinnati Reds,14,0.693,4.09,0.467
5,Pittsburgh Pirates,14,0.672,4.15,0.463
6,Los Angeles Dodgers,12,0.781,3.90,0.601
7,New York Yankees,12,0.762,3.74,0.570
8,Baltimore Orioles,12,0.750,3.94,0.555
9,Arizona Diamondbacks,12,0.777,4.62,0.546


Several teams—such as the Royals, Braves, and Twins—had the largest number of qualified hitters, indicating deep lineups. However, depth alone does not guarantee strong overall performance: among them, only Kansas City posted a clearly above-average win rate.

Top-performing teams like the Dodgers and Yankees had fewer qualified hitters, yet achieved higher OPS, better pitching metrics, and stronger win rates. This suggests that team success depends more on the quality of key hitters and pitching balance rather than simply the number of qualified hitters.

# Query 6 Ranking Teams by OPS

In [42]:
q6 = """
SELECT
    team_name,
    OPS,
    overall_win_rate,
    RANK() OVER (ORDER BY OPS DESC) AS ops_rank
FROM team_stats
ORDER BY ops_rank;
"""

df_q6 = run_sql(q6)
df_q6.head(10)

,team_name,OPS,overall_win_rate,ops_rank
0,Los Angeles Dodgers,0.781,0.601,1
1,Arizona Diamondbacks,0.777,0.546,2
2,New York Yankees,0.762,0.570,3
3,Philadelphia Phillies,0.750,0.582,4
4,Baltimore Orioles,0.750,0.555,4
5,San Diego Padres,0.744,0.570,6
6,Boston Red Sox,0.742,0.494,7
7,Houston Astros,0.740,0.543,8
8,New York Mets,0.734,0.524,9
9,Milwaukee Brewers,0.729,0.567,10


Teams with the highest OPS generally appear among the league’s top performers. Most clubs ranked in the top OPS tiers also post strong win rates, suggesting that offensive productivity is a key driver of overall success. However, exceptions exist—teams with strong OPS but weaker win rates—which highlights the importance of pitching and run prevention alongside hitting strength.

# Query 7: Teams with OPS Above League Average
Identify which teams exceed the league-average OPS and examine whether these higher-OPS teams also tend to have above-average win rates.

In [43]:
q7 = """
SELECT
    team_name,
    OPS,
    overall_win_rate
FROM team_stats
WHERE OPS >
    (SELECT AVG(OPS) FROM team_stats)   -- subquery for league-average OPS
ORDER BY OPS DESC;
"""

df_q7 = run_sql(q7)
df_q7


,team_name,OPS,overall_win_rate
0,Los Angeles Dodgers,0.781,0.601
1,Arizona Diamondbacks,0.777,0.546
2,New York Yankees,0.762,0.570
3,Philadelphia Phillies,0.750,0.582
4,Baltimore Orioles,0.750,0.555
5,San Diego Padres,0.744,0.570
6,Boston Red Sox,0.742,0.494
7,Houston Astros,0.740,0.543
8,New York Mets,0.734,0.524
9,Milwaukee Brewers,0.729,0.567


Teams that exceed the league-average OPS generally show stronger win rates, indicating a clear connection between offensive productivity and overall success. Although pitching and defense also contribute to performance, this result highlights that teams producing above-average offensive output tend to position themselves more favorably in the standings.

# Query 8: Rank teams by win rate

In [44]:
q8 = """
SELECT
    team_name,
    overall_win_rate,
    OPS,
    ERA,
    RANK() OVER (ORDER BY overall_win_rate DESC) AS win_rate_rank
FROM team_stats
ORDER BY win_rate_rank;
"""

df_q8 = run_sql(q8)
df_q8.head(10)   # top 10 teams by win rate

,team_name,overall_win_rate,OPS,ERA,win_rate_rank
0,Los Angeles Dodgers,0.601,0.781,3.90,1
1,Philadelphia Phillies,0.582,0.750,3.85,2
2,San Diego Padres,0.570,0.744,3.86,3
3,New York Yankees,0.570,0.762,3.74,4
4,Milwaukee Brewers,0.567,0.729,3.65,5
5,Baltimore Orioles,0.555,0.750,3.94,6
6,Cleveland Guardians,0.554,0.702,3.61,7
7,Arizona Diamondbacks,0.546,0.777,4.62,8
8,Houston Astros,0.543,0.740,3.74,9
9,New York Mets,0.524,0.734,3.96,10


This query ranks all teams by their overall win rate using a window function and then lists their OPS and ERA alongside. The top-ranked teams tend to combine above-average offensive production (higher OPS) with strong run prevention (lower ERA), confirming that both hitting and pitching quality jointly drive team success rather than offense alone.

# Query 9: Classify hitters into OPS quartiles
Use a window function to split qualified hitters into four OPS quartiles, and see which teams have many top-OPS hitters. This helps link individual offensive quality to team success.

In [45]:
q9 = """
SELECT
    Player,
    team,
    AB,
    OPS,
    overall_win_rate,
    NTILE(4) OVER (ORDER BY OPS DESC) AS OPS_quartile
FROM hitters
WHERE AB >= 300            -- focus on regularly playing hitters
ORDER BY OPS DESC;
"""

df_q9 = run_sql(q9)
df_q9.head(20)

,Player,team,AB,OPS,overall_win_rate,OPS_quartile
0,Aaron Judge,New York Yankees,559,1.159,0.570,1
1,Shohei Ohtani,Los Angeles Dodgers,636,1.036,0.601,1
2,Juan Soto,New York Yankees,576,0.988,0.570,1
3,Bobby Witt Jr.,Kansas City Royals,636,0.977,0.522,1
4,Yordan Alvarez,Houston Astros,552,0.959,0.543,1
5,Vladimir Guerrero Jr.,Toronto Blue Jays,616,0.940,0.458,1
6,Ketel Marte,Arizona Diamondbacks,504,0.932,0.546,1
7,Brent Rooker,Oakland Athletics,546,0.927,0.424,1
8,Marcell Ozuna,Atlanta Braves,606,0.924,0.517,1
9,Joc Pederson,Arizona Diamondbacks,367,0.908,0.546,1


This query ranks all regularly used hitters (AB ≥ 300) into four OPS quartiles using a window function. Hitters in quartile 1 represent the top offensive performers in the league, and many of them tend to belong to teams with higher overall win rates, suggesting that having truly elite bats—not just depth—plays an important role in driving team success.

# Query 10: Team OPS buckets and average win rate

In [46]:
q10 = """
-- Query 10: OPS buckets and average win rate
-- What: Group teams into OPS buckets and compare their average win rate.
-- How: Use a CTE to assign each team to an OPS bucket, then group by that bucket.
-- Why: To see whether stronger offensive performance (higher OPS) is associated
--      with higher overall win rates, which is useful for win-rate prediction.

WITH ops_bucket AS (
    SELECT
        team_name,
        OPS,
        overall_win_rate,
        CASE
            WHEN OPS >= 0.760 THEN 'High OPS'
            WHEN OPS BETWEEN 0.720 AND 0.759 THEN 'Mid OPS'
            ELSE 'Low OPS'
        END AS ops_group
    FROM team_stats
    WHERE OPS IS NOT NULL
      AND overall_win_rate IS NOT NULL
)
SELECT
    ops_group,
    COUNT(*) AS teams,
    ROUND(AVG(OPS), 3) AS avg_ops,
    ROUND(AVG(overall_win_rate), 3) AS avg_win_rate
FROM ops_bucket
GROUP BY ops_group
ORDER BY avg_ops DESC;
"""

df_q10 = run_sql(q10)
df_q10

,ops_group,teams,avg_ops,avg_win_rate
0,High OPS,3,0.773,0.572
1,Mid OPS,9,0.738,0.538
2,Low OPS,18,0.687,0.456


This query shows that teams in the “High OPS” bucket tend to have higher average win rates than “Mid OPS” and “Low OPS” teams. In other words, stronger offensive performance (as captured by OPS) is positively associated with winning percentage, reinforcing that OPS is a useful predictor to include in any win-rate model.

# Conclusion

The SQL analysis shows a consistent relationship between team performance and a small set of core metrics. League-wide results confirm a mild home-field advantage, which provides a baseline for comparing individual teams. At the team level, higher OPS, lower ERA, and positive run differential align strongly with higher win rates, suggesting that balanced performance on both offense and pitching is critical.

Player-level queries show that having many qualified hitters does not guarantee strong results; instead, teams with multiple top-tier OPS hitters tend to win more consistently. This indicates that elite offensive production, rather than lineup depth alone, is a key driver of success.

Overall, the SQL results imply that **OPS, ERA, and run differential** are the most reliable predictors of team quality and should form the basis of any win-rate model. These metrics capture how efficiently a team scores and prevents runs, and they consistently explain variation in win rates across the league in 2024.
